# air_passenger RAMP kit: data enrichment - _The Truth is out there..._
<i>Sylvain Tostain, 2017.</i>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display
pd.set_option('display.max_columns', None)

## Motivation
Let's import the dataset supplied into `data`, and detach our labels `y` (`log_PAX`) from the features supplied in the dataset in `X`.

In [2]:
data = pd.read_csv("../data/train.csv.bz2")

y = data.log_PAX
X = data.drop('log_PAX', axis=1)
X.DateOfDeparture = pd.to_datetime(X.DateOfDeparture, dayfirst=True)

display(X.dtypes)

DateOfDeparture     datetime64[ns]
Departure                   object
Arrival                     object
WeeksToDeparture           float64
std_wtd                    float64
dtype: object

Let's have a look at `y`.

In [3]:
display(y.describe())

count    8902.000000
mean       10.999048
std         0.993945
min         3.878108
25%        10.427055
50%        11.013357
75%        11.592888
max        14.007791
Name: log_PAX, dtype: float64

Let's have a look at `X` now...

In [4]:
display(X.head(5))

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


... and some basic statistics.

In [5]:
display(X.describe())

,WeeksToDeparture,std_wtd
count,8902.000000,8902.000000
mean,11.446469,8.617773
std,2.787140,2.139604
min,2.625000,2.160247
25%,9.523810,7.089538
50%,11.300000,8.571116
75%,13.240000,10.140521
max,23.163265,15.862216


Now, let's have a look at the timeframe at hand...

In [6]:
print(min(data['DateOfDeparture']))
print(max(data['DateOfDeparture']))

2011-09-01
2013-03-05


...and airports in the playground.

In [7]:
print("Factors in Departure:")
print(X.Departure.unique())
print("Total number of factors in Departure: {}".format(len(X.Departure.unique())))

print("\nFactors in Arrival:")
print(X.Arrival.unique())
print("Total number of factors in Arrival: {}".format(len(X.Arrival.unique())))

Factors in Departure:
['ORD' 'LAS' 'DEN' 'ATL' 'SFO' 'EWR' 'IAH' 'LAX' 'DFW' 'SEA' 'JFK' 'PHL'
 'MIA' 'DTW' 'BOS' 'MSP' 'CLT' 'MCO' 'PHX' 'LGA']
Total number of factors in Departure: 20

Factors in Arrival:
['DFW' 'DEN' 'LAX' 'ORD' 'SFO' 'MCO' 'LAS' 'CLT' 'MSP' 'EWR' 'PHX' 'DTW'
 'MIA' 'BOS' 'PHL' 'JFK' 'ATL' 'LGA' 'SEA' 'IAH']
Total number of factors in Arrival: 20


Needless to say that the data at hand is rather thin...

We definitely ought to seek additional external data likely to supply useful information.

From what we saw, we might consider joining on:
1. The <a href=http://www.iata.org/publications/Pages/code-search.aspx>IATA location id.</a> for `Departure` and/or ;
2. The same for `Arrival` and/or;
3. The `DateOfDeparture`.

If we look deeper, we could also make a join on the average `DateOfBooking` devised as follows:

In [8]:
from pandas.tseries.offsets import DateOffset

X['DateOfBooking'] = X.apply(lambda row: row.DateOfDeparture - DateOffset(weeks=row.WeeksToDeparture), axis=1)
X['DateOfBooking'] = X['DateOfBooking'].dt.date # This is to drop the meaningless hour
display(X.head(5))

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,DateOfBooking
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012-03-20
1,2012-09-10,LAS,DEN,14.285714,9.466734,2012-06-02
2,2012-10-05,DEN,LAX,10.863636,9.035883,2012-07-20
3,2011-10-09,ATL,ORD,11.480000,7.990202,2011-07-20
4,2012-02-21,DEN,SFO,11.450000,9.517159,2011-12-02


## Which kind of useful data can we think of?

We can think of a great diversity of information...

In order to focus on what is at stake, let's get back to our aim. According to the starting kit, _"the goal of this challenge is to predict the `log PAX` column"_ which is defined as _"related to the number of passengers (the actual number were changed for privacy reasons)"_.

If we rephrase, the issue at hand is to predict the number of passenger on a service between two airports on a certain date.

An underlying question is _what is a service between two airports_ or perhaps _what is a service between two cities/regions_, or even _what is likely to explain the need or motivations for people to go from city A to city B_.

Another question is _what is a city_.

Well, we all may have a few ideas, based upon our experience...

A _city_ is at least (not exhaustive, fortunately):
* Its inhabitants (how many people? how old? how rich?... in a few words its demography);
* Its community (how many households? how many students? how many graduates? how many rich/poor people?);
* Its economic activity (how many company? how much added value? how much employees? unemployed people?);
* Its local events maybe (superbowl? marathons? festivals?)

Why do people have to or wish to travel:
* To gather with friends and family to celebrate;
* To look for sea and sun in summer;
* To look for snow in winter;
* To attend meetings and events.

Therefore, a (non-exhaustive, unfortunately) list of features of interest could be:
* Latitudes and longitudes of airport/cities;
* General demographic and economic data, such as thoses published by the US Census;
* Federal and usually observed holidays;
* Maybe local temperatures weather?

Let's have a look as some data collected from the <a href=https://www.census.gov/data.html>US Census Bureau</a> for cities to which these airports belong (note that several airports are connected to New York) and coordinates of airports extracted from <a href=http://www.openstreetmap.org>Open street map</a>.

In [9]:
ext_data = pd.read_csv("../data_sources/airports_and_cities.csv")
ext_data.dtypes

Airport        object
Town           object
Latitude      float64
Longitude     float64
State          object
Pop_2010        int64
Pop_2016        int64
Pop_2015        int64
Age_median    float64
Companies       int64
Graduates     float64
Housings        int64
Income          int64
Foreigners      int64
Poverty       float64
Veterans        int64
dtype: object

In [10]:
ext_data.describe()

,Latitude,Longitude,Pop_2010,Pop_2016,Pop_2015,Age_median,Companies,Graduates,Housings,Income,Foreigners,Poverty,Veterans
count,20.000000,20.000000,2.000000e+01,2.000000e+01,2.000000e+01,20.000000,2.000000e+01,20.000000,2.000000e+01,20.000000,2.000000e+01,20.000000,20.000000
mean,37.390115,-92.340630,2.129988e+06,2.259051e+06,2.224876e+06,34.575000,2.678232e+05,82.730000,9.143744e+05,47567.100000,7.095767e+05,21.865000,52711.400000
std,5.663933,17.611717,2.761902e+06,2.863551e+06,2.826060e+06,2.092562,3.544467e+05,5.577124,1.138033e+06,15564.655405,1.097336e+06,5.778114,60585.658943
min,25.793400,-122.379000,2.383000e+03,6.930600e+04,3.999500e+04,31.600000,3.754400e+04,73.200000,3.159500e+04,5148.000000,1.043700e+04,13.200000,467.000000
25%,33.589625,-106.504500,5.428178e+05,5.928145e+05,6.385148e+05,33.175000,6.391175e+04,79.800000,2.448540e+05,43415.250000,1.041482e+05,19.525000,7931.000000
50%,38.740350,-86.163550,7.683295e+05,8.564690e+05,8.164500e+05,33.900000,1.013305e+05,82.150000,3.746020e+05,49362.000000,2.204400e+05,21.700000,32097.000000
75%,41.076975,-79.027625,2.248488e+06,2.403851e+06,2.342663e+06,35.800000,2.680120e+05,87.350000,9.934662e+05,53439.000000,5.872352e+05,23.325000,75547.000000
max,47.449000,-71.017800,8.175133e+06,8.537673e+06,8.426743e+06,39.400000,1.050911e+06,93.400000,3.422225e+06,81294.000000,3.138169e+06,40.300000,178967.000000


In [11]:
ext_data

,Airport,Town,Latitude,Longitude,State,Pop_2010,Pop_2016,Pop_2015,Age_median,Companies,Graduates,Housings,Income,Foreigners,Poverty,Veterans
0,ORD,Chicago,41.9796,-87.9045,Illinois,2695598,2704958,2717534,33.7,291007,82.3,1192544,48522,572066,22.3,8284
1,LAS,Las Vegas,36.0852,-115.1507,Nevada,583756,632912,605097,36.9,55856,83.3,250279,50202,127458,17.5,45145
2,DEN,Denver,39.8589,-104.6733,Colorado,600158,69306,649654,34.1,79097,86.1,294191,53637,10437,17.3,32249
3,ATL,Atlanta,33.6410,-84.4226,Georgia,420003,472522,448901,33.4,64593,89.0,228579,47527,32701,24.6,20009
4,SFO,San Francisco,37.6218,-122.3790,California,805235,870887,840763,38.5,116803,87.0,383676,81294,295417,13.2,26401
5,EWR,New York,40.6971,-74.1756,New York,8175133,8537673,8426743,35.8,1050911,80.3,3422225,53373,3138169,20.6,178967
6,IAH,Houston,29.9869,-95.3421,Texas,2099451,2303482,2217706,32.6,260347,76.7,927107,46187,632743,22.5,78064
7,LAX,Los Angeles,33.9425,-118.4090,California,3792621,3976322,3900794,34.9,497999,75.5,1436543,50205,1489926,22.1,97236
8,DFW,Dallas,32.8959,-97.0372,Texas,1197816,1317929,1260688,32.4,142658,74.5,533556,43781,305921,24.0,467
9,SEA,Seattle,47.4490,-122.3093,Washington,60866,704352,653017,35.8,83323,93.4,31595,70594,118225,13.5,31945


Some of these data have been easy to collect together with the others, but are unlikely to bear information regarding the issue at stake. We'll drop some of them.

In [12]:
ext_data_prep = ext_data.drop(['Town', 'State', 'Pop_2016', 'Pop_2015', 'Veterans'], axis=1)
ext_data_prep

,Airport,Latitude,Longitude,Pop_2010,Age_median,Companies,Graduates,Housings,Income,Foreigners,Poverty
0,ORD,41.9796,-87.9045,2695598,33.7,291007,82.3,1192544,48522,572066,22.3
1,LAS,36.0852,-115.1507,583756,36.9,55856,83.3,250279,50202,127458,17.5
2,DEN,39.8589,-104.6733,600158,34.1,79097,86.1,294191,53637,10437,17.3
3,ATL,33.6410,-84.4226,420003,33.4,64593,89.0,228579,47527,32701,24.6
4,SFO,37.6218,-122.3790,805235,38.5,116803,87.0,383676,81294,295417,13.2
5,EWR,40.6971,-74.1756,8175133,35.8,1050911,80.3,3422225,53373,3138169,20.6
6,IAH,29.9869,-95.3421,2099451,32.6,260347,76.7,927107,46187,632743,22.5
7,LAX,33.9425,-118.4090,3792621,34.9,497999,75.5,1436543,50205,1489926,22.1
8,DFW,32.8959,-97.0372,1197816,32.4,142658,74.5,533556,43781,305921,24.0
9,SEA,47.4490,-122.3093,60866,35.8,83323,93.4,31595,70594,118225,13.5


## What about holidays, why do we ignore them?

We said earlier that holidays may be relevant too. Nevertheless, we note from the `pandas time series` documentation that there is <a href=http://pandas.pydata.org/pandas-docs/stable/timeseries.html#holidays-holiday-calendars>some information available on holidays</a>.

In [13]:
from pandas.tseries.holiday import Holiday, USMemorialDay, AbstractHolidayCalendar, nearest_workday, MO
from datetime import datetime

cal = pd.tseries.holiday.USFederalHolidayCalendar()
cal.rules

[Holiday: New Years Day (month=1, day=1, observance=<function nearest_workday at 0x000000BF6D4F9BF8>),
 Holiday: Dr. Martin Luther King Jr. (month=1, day=1, offset=<DateOffset: kwds={'weekday': MO(+3)}>),
 Holiday: Presidents Day (month=2, day=1, offset=<DateOffset: kwds={'weekday': MO(+3)}>),
 Holiday: MemorialDay (month=5, day=31, offset=<DateOffset: kwds={'weekday': MO(-1)}>),
 Holiday: July 4th (month=7, day=4, observance=<function nearest_workday at 0x000000BF6D4F9BF8>),
 Holiday: Labor Day (month=9, day=1, offset=<DateOffset: kwds={'weekday': MO(+1)}>),
 Holiday: Columbus Day (month=10, day=1, offset=<DateOffset: kwds={'weekday': MO(+2)}>),
 Holiday: Veterans Day (month=11, day=11, observance=<function nearest_workday at 0x000000BF6D4F9BF8>),
 Holiday: Thanksgiving (month=11, day=1, offset=<DateOffset: kwds={'weekday': TH(+4)}>),
 Holiday: Christmas (month=12, day=25, observance=<function nearest_workday at 0x000000BF6D4F9BF8>)]

In [14]:
cal.holidays(datetime(2011, 9, 1), datetime(2013, 3, 5))

DatetimeIndex(['2011-09-05', '2011-10-10', '2011-11-11', '2011-11-24',
               '2011-12-26', '2012-01-02', '2012-01-16', '2012-02-20',
               '2012-05-28', '2012-07-04', '2012-09-03', '2012-10-08',
               '2012-11-12', '2012-11-22', '2012-12-25', '2013-01-01',
               '2013-01-21', '2013-02-18'],
              dtype='datetime64[ns]', freq=None)

We'll be able to handle holidays through pd.tseries.holiday. Therefore, we do not keep them for the preparation of our external dataset.

So far... job done ! We'll start from there for the time being.

Let's save our file and then try and extract our features...

In [15]:
ext_data_prep.to_csv('../submissions/iteration_3/external_data.csv', index=False)